In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import full_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# To create genre-specific npz files (required for randomizing splits)
#fileio.process_data_into_np_files()

# To create 27 random splits, each of size 32. (Except the last one, data/working/splits/split_26.npz is only size 5)
#fileio.create_random_splits() # Randomized with random.seed(1), so our splits will be identical

# To load a range of splits
#X, Y = fileio.load_splits(range(5)) # loads splits 0-4
#X.shape = (fileio.NUM_SAMPLES_GTZAN, 5 * 32)
#Y.shape = (24, 5 * 32)

# To generate chroma, aligned with the splits
#fileio.generate_chroma_splits_from_splits()

# To load a range of chroma splits
#X, Y = fileio.load_chroma_splits(range(5)) # loads chroma splits 0-4
#X.shape = (300, 12, 5 * 32)
#Y.shape = (24, 5 * 32)

In [3]:
def get_scores(targets, guesses):
    scores = []
    
    for idx in range(len(targets)):
        scores.append(get_score_single(targets[idx], guesses[idx, :]))
        
    return scores
    

In [4]:
def get_score_single(target, guess):
    scores = keys.get_vector_from_key(target) * 2
    return scores[keys.generate_one_hot_guess(guess.data)]

In [5]:
def load_torch_data_from_matrices(X, Y):
    X = torch.transpose(torch.from_numpy(X), 0, 2)
    Y = torch.from_numpy(keys.generate_one_hot_matrix(Y))

    X = X.float()
    Y = Y.long()
    
    return X, Y

In [14]:
import importlib

importlib.reload(full_model)

net = full_model.Net()
print(net)

X_test, Y_test = fileio.load_chroma_splits(range(20,27))
X_test, Y_test = load_torch_data_from_matrices(X_test, Y_test)

X, Y = fileio.load_chroma_splits(range(20))
X, Y = load_torch_data_from_matrices(X, Y)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

for epoch in range(5000):
    optimizer.zero_grad()
    
    Y_hat = net(X)
    loss = criterion(Y_hat, Y)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('epoch ' + str(epoch) + ' loss: ' + str(loss.item()))
        print('Train accuracy = ' + str(np.mean(get_scores(Y, Y_hat))))
        print('Test accuracy = ' + str(np.mean(get_scores(Y_test, net(X_test)))))
        
print('Done')

Net(
  (conv1): Conv1d(12, 32, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=32, out_features=24, bias=True)
)
epoch 0 loss: 3.1772894859313965
Train accuracy = 0.11062500000000001
Test accuracy = 0.11218274111675128
epoch 100 loss: 3.139582633972168
Train accuracy = 0.165625
Test accuracy = 0.15837563451776648
epoch 200 loss: 3.020251989364624
Train accuracy = 0.379375
Test accuracy = 0.3710659898477157
epoch 300 loss: 2.8972699642181396
Train accuracy = 0.5628125
Test accuracy = 0.5126903553299492
epoch 400 loss: 2.8207926750183105
Train accuracy = 0.593125
Test accuracy = 0.5294416243654823
epoch 500 loss: 2.785489797592163
Train accuracy = 0.5984375
Test accuracy = 0.5274111675126903
epoch 600 loss: 2.765796184539795
Train accuracy = 0.599375
Test accuracy = 0.5304568527918782
epoch 700 loss: 2.7526516914367676
Train accuracy = 0.60859375
Test accuracy = 0.5263959390862945
epoch 800 loss: 2.7429940700531006
Train accuracy = 0.61546875
Test accuracy = 0.5263959390862945

In [15]:
# Train accuracy
np.mean(get_scores(Y, net(X)))

0.6409375

In [16]:
# Test accuracy
X, Y = fileio.load_chroma_splits(range(20,27))
X, Y = load_torch_data_from_matrices(X, Y)

np.mean(get_scores(Y, net(X)))

0.5365482233502538